In [1]:
import numpy as np



from sklearn.metrics import accuracy_score
import random
import matplotlib.pyplot as plt


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from sklearn.decomposition import PCA
from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


from ultralytics import YOLO

from PIL import Image

random.seed(2023)

## MLP (Multi Layer perceptron)

This notebook details the training of the Multilayer perceptron algo on the dataset. The architecture we used was relatively simple with just 2 hidden layers

In [2]:
train = keras.utils.image_dataset_from_directory(
    directory='../../data/Faces updated',
    labels='inferred',
    label_mode='categorical',
    batch_size=140,
    image_size=(1000, 1000),
    validation_split = 0.2,
    subset = 'training',
    seed = 42,
    )

validation = keras.utils.image_dataset_from_directory(
    directory='../../data/Faces updated',
    labels='inferred',
    label_mode='categorical',
    batch_size=140,
    image_size=(1000, 1000),
    validation_split = 0.2,
    subset = 'validation',
    seed = 42,
    )

Found 164 files belonging to 3 classes.
Using 132 files for training.
Found 164 files belonging to 3 classes.
Using 32 files for validation.


In [3]:
#map to pca
#convert to numpy images and then perform PCA to extract the values
#convert images to numpy
train_processed_numpy = train.as_numpy_iterator()
validation_processed_numpy = validation.as_numpy_iterator()

X_train, y_train = train_processed_numpy.next()
X_val, y_val = validation_processed_numpy.next()

In [4]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1] * X_train.shape[2] * X_train.shape[3]))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1] * X_val.shape[2] * X_val.shape[3]))

In [5]:
pca = PCA(n_components = 50)

X_train_lower_dim = pca.fit_transform(X_train)

In [6]:
X_val_lower_dim = pca.transform(X_val)



In [19]:
# A very simple archi with just 2 hidden layers
model = keras.Sequential([
      
    layers.Dense(128, activation='relu', input_shape = (50,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(3, activation='softmax')  # num_classes is the number of facial expression classes
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train_lower_dim, y = y_train,                    
                    epochs=20)  # Adjust epochs as needed



Epoch 1/20


c:\Users\veena\OneDrive\Desktop\Nus\Y4S2\IS4242\Project Actual\moodify\emotion_detection\lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2964 - loss: 3857.6353  
Epoch 2/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4838 - loss: 1806.6992 
Epoch 3/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6528 - loss: 894.7250 
Epoch 4/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.6832 - loss: 516.6241  
Epoch 5/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8361 - loss: 196.6981 
Epoch 6/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8591 - loss: 133.4178 
Epoch 7/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9296 - loss: 24.8939 
Epoch 8/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9614 - loss: 10.3076
Epoch 9/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9746 - loss: 8.6165  
Epoch 10/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9571 - loss: 9.5601 
Epoch 11/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9817 - loss: 7.8621     
Epoch 12/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.9762 - loss: 

In [20]:
y_pred = model.predict(X_val_lower_dim)

y_pred_labels = y_pred.argmax(axis = 1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


In [21]:
accuracy_score(y_val.argmax(axis = 1), y_pred_labels)

0.6875

# Face cropping

In [2]:
#Read in the data
train = keras.utils.image_dataset_from_directory(
    directory='../../data/Faces updated',
    labels='inferred',
    label_mode='categorical',
    batch_size=140,
    image_size=(1000, 1000),
    validation_split = 0.2,
    subset = 'training',
    seed = 50,
   )

validation = keras.utils.image_dataset_from_directory(
    directory='../../data/Faces updated',
    labels='inferred',
    label_mode='categorical',
    batch_size=140,
    image_size=(1000, 1000),
    validation_split = 0.2,
    subset = 'validation',
    seed = 50,
  
    )

Found 164 files belonging to 3 classes.
Using 132 files for training.
Found 164 files belonging to 3 classes.
Using 32 files for validation.


In [3]:
#convert to numpy images and then perform PCA to extract the values
#convert images to numpy in order to facilitate the face cropping
train_processed_numpy = train.as_numpy_iterator()
validation_processed_numpy = validation.as_numpy_iterator()

X_train, y_train = train_processed_numpy.next()
X_val, y_val = validation_processed_numpy.next()

In [4]:
#convert the images to cropped faces
face_classifier = YOLO("../../app/src/face_detection/model/yolov8n-face.pt")

X_cropped_train = []
y_train_cropped = []

n_train = X_train.shape[0]
for i in range(n_train):
    face = X_train[i]
   
    faces = face_classifier.predict(face)
    boxes = faces[0].boxes.xyxy.tolist()
    if boxes:
        
        #extract the face based on the output from the YOLOv8 model        
        left, bottom, right, top = boxes[0]
        cropped_face = face[int(bottom):int(top), int(left):int(right)]
        #resize cropped face to a std shape, 100x100 for now but can adjust this
        pil_face = Image.fromarray(np.uint8(cropped_face))
        pil_face = pil_face.resize((200, 200))
        numpy_cropped_face = np.array(pil_face)
        #append this to the new list containing all cropped faces
        X_cropped_train.append(numpy_cropped_face)
        y_train_cropped.append(y_train[i])

X_val_cropped = []
y_val_cropped = []
n_val = X_val.shape[0]

for i in range(n_val):
    face = X_val[i]
    faces = face_classifier.predict(face)
    boxes = faces[0].boxes.xyxy.tolist()
    if boxes:
        
        #extract the face based on the output from the YOLOv8 model        
        left, bottom, right, top = boxes[0]
        cropped_face = face[int(bottom):int(top), int(left):int(right)]
        #resize cropped face to a std shape, 100x100 for now but can adjust this
        pil_face = Image.fromarray(np.uint8(cropped_face))
        pil_face = pil_face.resize((200, 200))
        numpy_cropped_face = np.array(pil_face)
        #append this to the new list containing all cropped faces
        X_val_cropped.append(numpy_cropped_face)
        y_val_cropped.append(y_val[i])


c:\Users\veena\OneDrive\Desktop\Nus\Y4S2\IS4242\Project Actual\moodify\emotion_detection\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0: 640x640 1 face, 153.1ms
Speed: 16.7ms preprocess, 153.1ms inference, 1237.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 102.2ms
Speed: 0.0ms preprocess, 102.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 114.9ms
Speed: 9.0ms preprocess, 114.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 78.6ms
Speed: 9.3ms preprocess, 78.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 81.4ms
Speed: 5.4ms preprocess, 81.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 82.9ms
Speed: 8.1ms preprocess, 82.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 78.1ms
Speed: 10.9ms preprocess, 78.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 81.3ms
Speed: 8.0ms preprocess, 81.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640

In [5]:
#change the variables to numpy arrays
X_cropped_train = np.array(X_cropped_train)

X_val_cropped = np.array(X_val_cropped)

print(X_cropped_train.shape)
print(X_val_cropped.shape)

#Flatten the RGB image into a flat tensor
X_cropped_train = X_cropped_train.reshape((X_cropped_train.shape[0], X_cropped_train.shape[1] * X_cropped_train.shape[2] * X_cropped_train.shape[3]))
X_val_cropped = X_val_cropped.reshape((X_val_cropped.shape[0], X_val_cropped.shape[1] * X_val_cropped.shape[2] * X_val_cropped.shape[3]))

(120, 200, 200, 3)
(29, 200, 200, 3)


In [6]:
pca = PCA(n_components = 50)

X_train_lower_dim = pca.fit_transform(X_cropped_train)

In [8]:
X_val_lower_dim = pca.transform(X_val_cropped)

In [7]:
# A very simple archi with just 2 hidden layers
model = keras.Sequential([
      
    layers.Dense(128, activation='relu', input_shape = (50,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(3, activation='softmax')  # num_classes is the number of facial expression classes
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train_lower_dim, y = np.array(y_train_cropped),                    
                    epochs=20)  # Adjust epochs as needed



Epoch 1/20


c:\Users\veena\OneDrive\Desktop\Nus\Y4S2\IS4242\Project Actual\moodify\emotion_detection\lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.3406 - loss: 1389.1788  
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3483 - loss: 585.2892 
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.5165 - loss: 391.1224  
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6615 - loss: 236.5459 
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 167us/step - accuracy: 0.7198 - loss: 105.9903
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7692 - loss: 74.2471 
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8381 - loss: 31.6282 
Epoch 8/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9265 - loss: 11.2160
Epoch 9/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9604 - loss: 4.7899 
Epoch 10/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9775 - loss: 2.2807     
Epoch 11/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9852 - loss: 1.6458 
Epoch 12/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9852 - loss: 1

In [10]:
#get the accuracy of the cropped images
y_pred = model.predict(X_val_lower_dim)

accuracy_score(np.array(y_val_cropped).argmax(1), y_pred.argmax(1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


0.6551724137931034